In [11]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import matplotlib.pyplot as plt
import numpy as np

In [14]:
#network
b = 1. #initial translation
a = 1. #initial slope

#validation set
m = 1000 #size
u, v = -10., 10. #domain [u, v]

#loss and optimizer
batchsize = 10
lr = 1e-3
iterations = 150 #number of gradient descents
criterion = nn.MSELoss() #loss function

In [15]:
#network
class Net(nn.Module):

    def __init__(self):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(1, 2)
        self.fc1.weight.data=torch.tensor([[1.], [1.]])
        self.fc1.bias.data=torch.tensor([0., -b])
        self.fc2 = nn.Linear(2, 1)
        self.fc2.weight.data=torch.tensor([[a, -a]])
        self.fc2.bias.data=torch.tensor([0.])

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x

#optimization
def optimize(iterations, net):
    
    #validation set
    x_val = torch.linspace(u, v, m).view(-1,1)
    y_val = torch.zeros(m, 1)
    
    optimizer = optim.SGD(net.parameters(), lr=lr, momentum=0.9)
    parameter = [net.parameters()]
    
    labels = torch.zeros(batchsize, 1)
    for i in range(iterations):  

            inputs = torch.FloatTensor(batchsize, 1).uniform_(u, v)

            # zero the parameter gradients
            optimizer.zero_grad()

            # forward + backward + optimize
            loss = criterion(net(inputs), labels)
            loss.backward()
            optimizer.step()

            # print statistics
            if i % 30 == 29:    # print every 10 mini-batches
                loss = criterion(net(x_val), y_val)
                error = torch.max(abs(net(x_val) - y_val))
                print('iteration: %5d   loss: %.3f   max error: %.3f' %(i + 1, loss, error))
                parameter.append(net.parameters())

    print('Finished Training')
    return parameter

In [18]:
#network
net=Net()
parameter=optimize(iterations)

iteration:    30   loss: 0.092   max error: 0.767
iteration:    60   loss: 0.053   max error: 0.620
iteration:    90   loss: 0.042   max error: 0.515
iteration:   120   loss: 0.031   max error: 0.444
iteration:   150   loss: 0.027   max error: 0.398
Finished Training


In [21]:
#plotting the predicted function

delta = 0 #increase interval by delta

x_plt = torch.linspace(u-delta, v+delta, m).view(-1,1)
with torch.no_grad():
    y_plt = net(x_plt)

plt.plot(x_plt.numpy(), y_plt.numpy())
plt.show()

AttributeError: module 'matplotlib.colors' has no attribute 'to_rgba'

In [208]:
#plotting the parameters

print(parameter[1])

<generator object Module.parameters at 0x7f2934a2c570>
